In [2]:
from collections import deque
from dataclasses import dataclass
from math import gcd

# Advent of Code 2024 Day 13 (Claw Contraption)

## Part 1

In [3]:
@dataclass(slots=True)
class ClawMachine:
    a: tuple
    b: tuple
    prize: tuple
    pos: tuple = (0, 0)
    spent: int = 0

    def push_button(self, button: str):
        try:
            match button:
                case "a":
                    self.pos = (self.pos[0] + self.a[0], self.pos[1] + self.a[1])
                    self.spent += 3
                case "b":
                    self.pos = (self.pos[0] + self.b[0], self.pos[1] + self.b[1])
                    self.spent += 1
                case _:
                    raise ValueError("invalid button pushed")
        except ValueError as e:
            print(f"an error has occured: {e}")

    def is_reachable(self):
        def gcd2(a, b):
            return gcd(abs(a), abs(b))

        gcd_x = gcd2(self.a[0], self.b[0])
        gcd_y = gcd2(self.a[1], self.b[1])

        dx, dy = self.prize[0] - self.pos[0], self.prize[1] - self.pos[1]
        return dx % gcd_x == 0 and dy % gcd_y == 0

    def find_minimum_cost(self):
        if not self.is_reachable():
            return 0

        queue = deque([(self.pos, self.spent, 0, 0)])
        visited = set()
        visited.add((self.pos, 0, 0))

        while queue:
            current_pos, current_cost, a_count, b_count = queue.popleft()

            if current_pos == self.prize:
                return current_cost

            if a_count < 100:
                original_pos = self.pos
                original_spent = self.spent
                self.pos = current_pos
                self.spent = current_cost
                self.push_button("a")
                if (self.pos, a_count + 1, b_count) not in visited:
                    visited.add((self.pos, a_count + 1, b_count))
                    queue.append((self.pos, self.spent, a_count + 1, b_count))
                self.pos = original_pos
                self.spent = original_spent

            if b_count < 100:
                original_pos = self.pos
                original_spent = self.spent
                self.pos = current_pos
                self.spent = current_cost
                self.push_button("b")
                if (self.pos, a_count, b_count + 1) not in visited:
                    visited.add((self.pos, a_count, b_count + 1))
                    queue.append((self.pos, self.spent, a_count, b_count + 1))
                self.pos = original_pos
                self.spent = original_spent

        return 0


In [7]:
machines = []

with open("data/claw.txt", "r") as file:
    mach_data = [m.split("\n") for m in file.read().strip().split("\n\n")]
    for machine in mach_data:
        a = machine[0].lstrip("Button A: X+").replace("Y+", "")
        b = machine[1].lstrip("Button A: X+").replace("Y+", "")
        prize = machine[2].lstrip("Prize: X=").replace("Y=", "")

        machines.append(ClawMachine(
            a = tuple([int(coord) for coord in a.split(",")]),
            b = tuple([int(coord) for coord in b.split(",")]),
            prize = tuple([int(coord) for coord in prize.split(",")])
        ))

lowest = sum([machine.find_minimum_cost() for machine in machines])
lowest

36838

In [8]:
with open("answer.txt", "w") as file:
    file.writelines([str(lowest)])